In [1]:
from DTG.classes.LLM import *
from DTG.prompts import prompt_funcs
import json, time

In [2]:
INPUT_INFO = 'NPC personality : lazy and shy . NPC mood : confused and amused. Convesation features : dancing, great move, tripping'

In [3]:
def get_dialogue_json(input_info):
    t_0 = time.time()
    starting_time = time.time()
    performance_dict = {
        'tags' : 0,
        'abstract' : 0,
        'endings' : 0,
        'tree' : 0,
        'formatted_json' : 0
    }
    llm = LLM()
    print(f'Generating tags.')
    tags_by_llm = llm.send_message_for_code(prompt_funcs.tag_prompt(input_info), model='gpt-4o-mini')
    print(f'Tags generated in {time.time() - starting_time} seconds.')
    performance_dict['tags'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Generating abstract.')
    abstract = llm.send_message_for_code(prompt_funcs.abstract_prompt(tags_by_llm, input_info))
    print(f'Abstract generated in {time.time() - starting_time} seconds.')
    performance_dict['abstract'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Generating possible endings.')
    endings_by_llm = llm.send_message_for_code(prompt_funcs.possible_ending_prompt(abstract), model='o1-mini')
    print(f'Possible endings generated in {time.time() - starting_time} seconds.')
    performance_dict['endings'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Generating dialogue tree.')
    tree_by_llm = llm.send_message_for_code(prompt_funcs.dialogue_prompt(abstract, endings_by_llm, tags_by_llm), model='o1-preview')
    print(f'Dialogue tree generated in {time.time() - starting_time} seconds.')
    performance_dict['tree'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Formatting JSON.')
    formatted_tree = llm.send_message(prompt_funcs.format_prompt(tree_by_llm), model='gpt-4o-mini')
    print(f'JSON formatted in {time.time() - starting_time} seconds.')
    performance_dict['formatted_json'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Total time taken : {time.time() - t_0} seconds.')
    print(f'Performance dict : {performance_dict}')
    formatted_tree_json = json.loads(formatted_tree)
    #formatted_endings_json = json.loads(endings_by_llm)
    return formatted_tree_json, endings_by_llm


In [4]:
#tree, endings = get_dialogue_json(INPUT_INFO)

In [5]:
from DTG.classes.OutputClasses import Tree, PossibleEndings

In [6]:
def get_dialogue_json(input_info):
    t_0 = time.time()
    starting_time = time.time()
    performance_dict = {
        'tags' : 0,
        'abstract' : 0,
        'endings' : 0,
        'tree' : 0
    }
    llm = LLM()
    print(f'Generating tags.')
    tags_by_llm = llm.send_message_for_code(prompt_funcs.tag_prompt(input_info), model='gpt-4o-mini')
    print(f'Tags generated in {time.time() - starting_time} seconds.')
    performance_dict['tags'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Generating abstract.')
    abstract = llm.send_message_for_code(prompt_funcs.abstract_prompt(tags_by_llm, input_info))
    print(f'Abstract generated in {time.time() - starting_time} seconds.')
    performance_dict['abstract'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Generating possible endings.')
    endings_by_llm = llm.send_message_for_format(prompt_funcs.possible_ending_prompt(abstract), PossibleEndings ,model='gpt-4o')
    print(f'Possible endings generated in {time.time() - starting_time} seconds.')
    performance_dict['endings'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Generating dialogue tree.')
    tree_by_llm = llm.send_message_for_format(prompt_funcs.dialogue_prompt(abstract, endings_by_llm, tags_by_llm), format=Tree,model='gpt-4o')
    print(f'Dialogue tree generated in {time.time() - starting_time} seconds.')
    performance_dict['tree'] = time.time() - starting_time
    starting_time = time.time()
    print(f'Total time taken : {time.time() - t_0} seconds.')
    print(f'Performance dict : {performance_dict}')
    #formatted_endings_json = json.loads(endings_by_llm)
    return tree_by_llm, endings_by_llm


In [7]:
tree, endings = get_dialogue_json(INPUT_INFO)

Generating tags.
Tags generated in 7.584046125411987 seconds.
Generating abstract.
Abstract generated in 2.453310966491699 seconds.
Generating possible endings.
Possible endings generated in 1.4535679817199707 seconds.
Generating dialogue tree.
Dialogue tree generated in 33.83434271812439 seconds.
Total time taken : 45.325748920440674 seconds.
Performance dict : {'tags': 7.584151029586792, 'abstract': 2.45344614982605, 'endings': 1.4536759853363037, 'tree': 33.83446907997131}


In [8]:
with open('output.json', 'w') as file:
    json.dump(json.loads(tree), file, indent=4)

In [13]:
with open('PossibleRndings.json', 'w') as file:
    json.dump(json.loads(endings), file, indent=4)

In [9]:
import requests, json

In [10]:
url = 'http://127.0.0.1:5000'

In [14]:
url = "https://us-central1-xg-app-43806.cloudfunctions.net/generate_tags"

payload = json.dumps({
  "input_info": "NPC personality : lazy and shy . NPC mood : confused and amused. Convesation features : dancing, great move, tripping"
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

In [15]:
tags = response.json()['tags']

In [16]:
tags

'#greeting #question #confused #polite #amused  \n#question #answer #supportive #informal #amused  \n#statement #exclamation #excited #energetic #friendly  \n#request #statement #disappointed #sympathetic #polite  \n#exclamation #complaint #disgusted #confused #rude  \n#joke #exclamation #funny #amused #polite  \n#apology #statement #regretful #polite #sincere  \n#thanks #statement #friendly #positive #supportive  \n#suggestion #request #helpful #friendly #calm  \n#goodbye #statement #neutral #polite #unfriendly'

In [17]:
url = "https://us-central1-xg-app-43806.cloudfunctions.net/generate_abstract"
payload = json.dumps({
    "input_info" : INPUT_INFO,
    "tags" : tags
})

response = requests.request("POST", url, headers=headers, data=payload)

In [18]:
abstract = response.json()['abstract']
abstract

"As the player steps into the bustling spirit concourse, they are greeted by a somewhat lazy but shy spirit who slowly sways in a daze, exuding a blend of confusion and amusement about the afterlife. Their conversation kicks off with an awkward greeting, where the spirit sheepishly questions the player about the funky dance vibes permeating the air, only to trip over their own feet in an enthusiastic attempt to demonstrate their moves. The player chuckles, trying to support the spirit's carefree attitude while secretly wishing they could whip them into shape for the dance floor of the afterlife. Amidst the laughter, the spirit rambles on about how they wished for a spa facility instead, expressing slight disappointment but with a warm, friendly tone. As the playful banter escalates, the spirit humorously complains about the other spirits who don't dance as well, offering a cheeky joke that leaves both in stitches before they finally part ways, the spirit awkwardly swaying off while the

In [19]:
url = "https://us-central1-xg-app-43806.cloudfunctions.net/generate_possible_endings"
payload = payload = json.dumps({
    "input_info" : INPUT_INFO,
    "tags" : tags,
    "abstract" : abstract
})
response = requests.request("POST", url, headers=headers, data=payload)

In [20]:
endings = response.json()['endings']
endings

'{"positive":{"parameter_tag":"Mood, Currency","value":"{\\"mood\\" : 0.3, \\"currency\\" : 50}"},"neutral":{"parameter_tag":"Mood, Restlessness","value":"{\\"mood\\" : 0.0, \\"restlessness\\" : 0.0}"},"negative":{"parameter_tag":"Mood, Restlessness","value":"{\\"mood\\" : -0.4, \\"restlessness\\" : 0.2}"}}'

In [23]:
url = "https://us-central1-xg-app-43806.cloudfunctions.net/generate_dialogue_tree"
payload = payload = json.dumps({
    "input_info" : INPUT_INFO,
    "tags" : tags,
    "abstract" : abstract,
    "endings" : endings
})
response = requests.request("POST", url, headers=headers, data=payload)

In [ ]:
tree_json = response.json()['tree']


'{"nodes":[{"id":"node_1","npc_dialogue":"Whoa there, fancy feet! Did you feel those funky dance vibes, or am I just really in the groove now?","tag_of_node":"#greeting #question #confused #polite #amused","player_dialogue_choices":[{"player_dialogue_choice":"Maybe it\'s the afterlife clubbing scene kicking in! Need a partner?","next_node":"node_2"},{"player_dialogue_choice":"Not bad! Are you sure you didn\'t trip over an afterlife rug?","next_node":"node_3"}],"end_node_effect":"neutral","is_end_node":false},{"id":"node_2","npc_dialogue":"Haha! Partner in life and the afterlife, huh? But I bet you can\'t beat my moves!","tag_of_node":"#question #answer #supportive #informal #amused","player_dialogue_choices":[{"player_dialogue_choice":"Challenge accepted! Let\'s see who can moonwalk better than MJ.","next_node":"node_4"},{"player_dialogue_choice":"I\'ll let you take the floor. I\'m more of a spa-type spirit myself!","next_node":"node_5"}],"end_node_effect":"neutral","is_end_node":false